In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

from pathlib import Path

import logging
logging.basicConfig(level=logging.INFO)

import numpy

In [8]:
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot

from structlib.visualize import show_atoms_mpl_3d
from structlib.vec import split_arr
from structlib.transform import Transform

def plot_corners(ax: Axes3D, transform: Transform, fmt='.-k', alpha=1, markersize=8, label=None, **kwargs):
    corners = numpy.stack(list(map(numpy.ravel, numpy.meshgrid((0, 1), (0, 1), (0, 1), indexing='ij'))), axis=-1)
    corners = transform @ corners
    faces = [
        numpy.array([
            corners[(val*2**axis + v1*2**((axis+1) % 3) + v2*2**((axis+2) % 3))]
            for (v1, v2) in [(0, 0), (0, 1), (1, 1), (1, 0), (0, 0)]
        ])
        for axis in (0, 1, 2)
        for val in (0, 1)
    ]
    for i, face in enumerate(faces):
        ax.plot3D(*split_arr(face, axis=-1), fmt, alpha=alpha, markersize=markersize, label=label if i == 0 else None, **kwargs)

In [3]:
from structlib.cell import cell_to_ortho, Cell

mono_ortho = cell_to_ortho([3., 4., 5.], [numpy.pi/2., numpy.pi/2., 1.8])
    #.rotate([0., 0., 1.], numpy.pi/2.)
cell = Cell.from_ortho(mono_ortho, n_cells=[2, 3, 5])

In [9]:
fig = pyplot.figure()
ax: Axes3D = fig.add_axes([0., 0., 1., 1.], axes_class=Axes3D, proj_type='ortho')

plot_corners(ax, cell.get_transform(frame_from='cell_box'), '.:k', label='cell_box')
plot_corners(ax, cell.get_transform(frame_from='ortho_box'), '.:r', label='ortho_box')
plot_corners(ax, cell.get_transform(frame_from='cell_frac'), '.-k', label='cell_frac')
plot_corners(ax, cell.get_transform(frame_from='ortho_frac'), '.-r', label='ortho_frac')

ax.legend()

In [17]:
cell.get_transform(frame_from='ortho_box') @ [0., 1., 0.]